# Construct dataset for MIMIC analysis

Note:  MIMIC-ED should be downloaded at “data/ed" folder (please see the README for more instructions).  

This code is adapted from https://github.com/rmovva?tab=repositories, which was in turn adapted from https://github.com/nliulab/mimic4ed-benchmark.


## Prepare Python library and raw data

In [1]:
# reload packages from notebook whenever needed
%load_ext autoreload
%autoreload 2
import os

import sys
sys.path.append('/data/cb/shuvom/funnels/repo/data_cleaning')

from helpers import *
from medcode_utils import comorbidity, extract_icd_list
from paths import mimic_iv_path, data_path

In [2]:
# Define auxillary paths/files
# 'ed' subfolder should also be added to mimic_iv_path
mimic_iv_core_path = os.path.join(mimic_iv_path, 'hosp')
mimic_iv_hosp_path = os.path.join(mimic_iv_path , 'hosp')   
mimic_iv_icu_path = os.path.join(mimic_iv_path, 'icu')
mimic_iv_ed_path = os.path.join(mimic_iv_path, 'ed')

# Use gzipped filepaths for auxiliary MIMIC data
icu_filename_dict = {"chartevents":"chartevents.csv.gz", "datetimeevents":"datetimeevents.csv.gz","d_items":"d_items.csv.gz","icustays":"icustays.csv.gz","inputevents":"inputevents.csv.gz","outputevents":"outputevents.csv.gz","procedureevents":"procedureevents.csv.gz"}
core_filename_dict = {"patients":"patients.csv.gz", "admissions":"admissions.csv.gz", "transfers":"transfers.csv.gz"}
hosp_filename_dict = {"d_hcpcs":"d_hcpcs.csv.gz","d_icd_diagnoses":"d_icd_diagnoses.csv.gz","d_labitems":"d_labitems.csv.gz","emar":"emar.csv.gz","hcpcsevents":"hcpcsevents.csv.gz","microbiologyevents":"microbiologyevents.csv.gz","poe":"poe.csv.gz","prescriptions":"prescriptions.csv.gz","services":"services.csv.gz","diagnoses_icd":"diagnoses_icd.csv.gz","d_icd_procedures":"d_icd_procedures.csv.gz","drgcodes":"drgcodes.csv.gz","emar_detail":"emar_detail.csv.gz","labevents":"labevents.csv.gz","pharmacy":"pharmacy.csv.gz","poe_detail":"poe_detail.csv.gz","procedures_icd":"procedures_icd.csv.gz"}
ed_filename_dict = {'diagnosis':'diagnosis.csv.gz', 'edstays':'edstays.csv.gz',  'medrecon':'medrecon.csv.gz',  'pyxis':'pyxis.csv.gz',  'triage':'triage.csv.gz',  'vitalsign':'vitalsign.csv.gz'}

complaint_dict = {"chiefcom_chest_pain" : "chest pain", "chiefcom_abdominal_pain" : "abdominal pain|abd pain", 
"chiefcom_headache" : "headache|lightheaded", "chiefcom_shortness_of_breath" : "breath", "chiefcom_back_pain" : "back pain", "chiefcom_cough" : "cough", 
"chiefcom_nausea_vomiting" : "nausea|vomit", "chiefcom_fever_chills" : "fever|chill", "chiefcom_syncope" :"syncope", "chiefcom_dizziness" : "dizz"}

# Defining parameters for the outcome variables
icu_transfer_timerange = 12 # Considered critical outcome if patient is transferred to ICU within 12 hours of ED visit
next_ed_visit_timerange = 3 # Considered an ED revisit if patient returns to ED within 3 days of discharge


## Load raw data tables through pandas library

In [ ]:
## Reading main tables
df_edstays = read_edstays_table('/data/cb/scratch/sophia/rsidata/physionet.org/files/mimic-iv-ed/2.2/ed/edstays.csv.gz')
df_patients = read_patients_table('/data/cb/scratch/sophia/rsidata/physionet.org/files/mimiciv/2.2/hosp/patients.csv.gz')
df_admissions = read_admissions_table('/data/cb/scratch/sophia/rsidata/physionet.org/files/mimiciv/2.2/hosp/admissions.csv.gz')
df_icustays = read_icustays_table('/data/cb/scratch/sophia/rsidata/physionet.org/files/mimiciv/2.2/icu/icustays.csv.gz')
df_triage = read_triage_table('/data/cb/scratch/sophia/rsidata/physionet.org/files/mimic-iv-ed/2.2/ed/triage.csv.gz')
df_vitalsign = read_vitalsign_table('/data/cb/scratch/sophia/rsidata/physionet.org/files/mimic-iv-ed/2.2/ed/vitalsign.csv.gz')
df_pyxis = read_pyxis_table('/data/cb/scratch/sophia/rsidata/physionet.org/files/mimic-iv-ed/2.2/ed/pyxis.csv.gz')
df_medrecon = read_pyxis_table('/data/cb/scratch/sophia/rsidata/physionet.org/files/mimic-iv-ed/2.2/ed/medrecon.csv.gz')

## Read data here for ICD
df_diagnoses = read_diagnoses_table('/data/cb/scratch/sophia/rsidata/physionet.org/files/mimiciv/2.2/hosp/diagnoses_icd.csv.gz')
df_transfers = pd.read_csv('/data/cb/scratch/sophia/rsidata/physionet.org/files/mimiciv/2.2/hosp/transfers.csv.gz', compression='gzip')

## Add demographics and outcomes

In [ ]:
## Merging patients -> merging admissions -> merging triage -> master
df_main = merge_edstays_patients_on_subject(df_edstays, df_patients)
df_main = merge_edstays_admissions_on_subject(df_main, df_admissions)

In [ ]:
transfers_with_hadm = df_transfers[df_transfers['hadm_id'].notna()]

# Group by hadm_id to analyze patient journeys
patient_journeys = transfers_with_hadm.groupby('hadm_id')
# print(patient_journeys)

# Find patients who meet our criteria:
# 1. First admitted to non-ICU
# 2. Later transferred to ICU
icu_transfers = []

import tqdm
for (hadm_id, journey) in tqdm.tqdm(patient_journeys):
    journey = journey.sort_values('intime')
    
    # Check if first admission is to non-ICU
    first_admission = journey[journey['eventtype'] == 'admit'].iloc[0] if not journey[journey['eventtype'] == 'admit'].empty else None
    
    if first_admission is not None and 'ICU' not in str(first_admission['careunit']):
        # Check if there's a later transfer to ICU
        later_transfers = journey[(journey['eventtype'] == 'transfer') & 
                                 (journey['intime'] > first_admission['intime'])]
        # print("later_transfers", later_transfers)
        
        icu_transfers_in_journey = later_transfers[later_transfers['careunit'].str.contains('ICU', na=False)]

        
        if not icu_transfers_in_journey.empty:
            # Add all relevant transfers to our list
            icu_transfers.append(journey)

100%|██████████| 431231/431231 [06:38<00:00, 1081.47it/s]


In [ ]:
## Adding age, mortality and ICU transfer outcome
df_main = add_age(df_main)
df_main = add_inhospital_mortality(df_main)
df_main = add_ed_los(df_main)
df_main = add_outcome_icu_transfer(df_main, df_icustays, icu_transfer_timerange)
df_main['outcome_hospitalization'] = ~pd.isnull(df_main['hadm_id'])
df_main['mortality'] = df_main['outcome_inhospital_mortality']
df_main['icuized'] = df_main[''.join(['outcome_icu_transfer_', str(icu_transfer_timerange), 'h'])]

# Sort main table for further processing
df_main = df_main.sort_values(['subject_id', 'intime']).reset_index()

# Filling subjects NA ethnicity, takes ~17s
df_main = fill_na_ethnicity(df_main)

In [ ]:
# Identify hadm_ids from icu_transfers (i.e., those who went to ICU after hospitalization)
icu_transfer_hadm_ids = set()
for journey in icu_transfers:
    if 'hadm_id' in journey.columns:
        icu_transfer_hadm_ids.update(journey['hadm_id'].unique())

df_main['icu_direct'] = 0
mask_icuized = df_main['icuized'] == True

# ICUized + in icu_transfe"r list -> icu_direct=0
df_main.loc[mask_icuized & df_main['hadm_id'].isin(icu_transfer_hadm_ids), 'icu_direct'] = 0

# ICUized + NOT in icu_transfer list -> icu_direct=1
df_main.loc[mask_icuized & ~df_main['hadm_id'].isin(icu_transfer_hadm_ids), 'icu_direct'] = 1

In [ ]:
df_main['icu_direct'].value_counts()

icu_direct
False    425087
Name: count, dtype: int64

In [ ]:

hospitalized_and_icu = df_main[(df_main['outcome_hospitalization'] == True) & (df_main['icuized'] == True)]
df_main['icu_direct'] = ((df_main['outcome_hospitalization'] == False) & (df_main['icuized'] == True))

# New column: stage
# 1 - not hospitalized and not icuized
# 2 - hospitalized but not icuized
# 3 - icuized
df_main['stage'] = 1
df_main.loc[(df_main['outcome_hospitalization'] == True) & (df_main['icuized'] == False), 'stage'] = 2
df_main.loc[df_main['icuized'] == True, 'stage'] = 3

df_main['stage'].value_counts()

stage
1    222071
2    176427
3     26589
Name: count, dtype: int64

## Triage Information

In [ ]:
# Merging with triage table, cols of form triage_*
df_main = merge_edstays_triage_on_subject(df_main, df_triage)

In [ ]:
# Encoding 10 chief complaints, chiefcom_*
df_main = encode_chief_complaints(df_main, complaint_dict)

## Remove outliers

In [ ]:
from helpers import vitals_valid_range

df_main = convert_temp_to_celsius(df_main)
display_outliers_count(df_main, vitals_valid_range)

,variable,< outlier_low,"[outlier_low, valid_low)","[valid_low, valid_high]","(valid_high, outlier_high]",> outlier_high
0,triage_temperature,479,29,401155,0,9
1,triage_heartrate,0,0,407990,0,7
2,triage_resprate,0,0,404731,0,3
3,triage_o2sat,0,0,404444,6,41
4,triage_sbp,0,0,406780,0,16
5,triage_dbp,0,0,405601,0,395
6,triage_pain,0,0,386851,0,11205
7,triage_acuity,0,0,418100,0,0


In [ ]:
features_to_normalize = [
    'triage_temperature',
    'triage_heartrate',
    'triage_resprate',
    'triage_o2sat',
    'triage_sbp',
    'triage_dbp',
    'triage_pain',
    'triage_acuity'
]

for feat in features_to_normalize:
    mean = df_main[feat].mean(skipna=True)
    std = df_main[feat].std(skipna=True)
    z_col = f"{feat}_z"
    sq_col = f"{feat}_sq"
    # z-score
    df_main[z_col] = (df_main[feat] - mean) / std
    # impute mean for missing values
    df_main[z_col] = df_main[z_col].fillna(0)  # mean of z-score is 0
    # add squared column
    df_main[sq_col] = df_main[z_col] ** 2

df_main['intercept'] = 1

In [ ]:
granular_to_coarse = {
    'HISPANIC/LATINO - PUERTO RICAN': 'HISPANIC OR LATINO', 
    'HISPANIC/LATINO - DOMINICAN': 'HISPANIC OR LATINO', 
    'HISPANIC/LATINO - GUATEMALAN': 'HISPANIC OR LATINO', 
    'HISPANIC/LATINO - SALVADORAN': 'HISPANIC OR LATINO', 
    'HISPANIC/LATINO - MEXICAN': 'HISPANIC OR LATINO', 
    'HISPANIC/LATINO - COLUMBIAN': 'HISPANIC OR LATINO', 
    'HISPANIC/LATINO - HONDURAN': 'HISPANIC OR LATINO', 
    'HISPANIC/LATINO - CUBAN': 'HISPANIC OR LATINO',
    'HISPANIC/LATINO - CENTRAL AMERICAN': 'HISPANIC OR LATINO',
    'SOUTH AMERICAN': 'HISPANIC OR LATINO',
    'ASIAN - CHINESE': 'ASIAN',
    'ASIAN - SOUTH EAST ASIAN': 'ASIAN',
    'ASIAN - ASIAN INDIAN': 'ASIAN',
    'ASIAN - KOREAN': 'ASIAN',
    'WHITE - OTHER EUROPEAN': 'WHITE',
    'WHITE - RUSSIAN': 'WHITE',
    'WHITE - EASTERN EUROPEAN': 'WHITE',
    'WHITE - BRAZILIAN': 'WHITE',
    'PORTUGUESE': 'WHITE',
    'BLACK/CAPE VERDEAN': 'BLACK/AFRICAN AMERICAN',
    'BLACK/CARIBBEAN ISLAND': 'BLACK/AFRICAN AMERICAN',
    'BLACK/AFRICAN': 'BLACK/AFRICAN AMERICAN',
    'AMERICAN INDIAN/ALASKA NATIVE': 'OTHER',
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'OTHER',
    'MULTIPLE RACE/ETHNICITY': 'OTHER',
    'UNKNOWN': 'OTHER',
    'PATIENT DECLINED TO ANSWER': 'OTHER',
    'UNABLE TO OBTAIN': 'OTHER',
}

'''
Input: patient reported race recorded in MIMIC-IV
Output: 
If the reported race is granular, then return its coarse category;
else, return the reported (coarse) race.

The goal is to construct a coarse race column for all patients.
'''
def coarsen_race(race):
    if race in granular_to_coarse: return granular_to_coarse[race]
    else: return race

df_main.insert(7, 'coarse_race', df_main['race'].apply(coarsen_race))
# df_main.rename(columns={'race': 'granular_race'}, inplace=True)
df_main['coarse_race'].value_counts()

coarse_race
WHITE                     247459
BLACK/AFRICAN AMERICAN     92998
HISPANIC OR LATINO         35574
OTHER                      30528
ASIAN                      18528
Name: count, dtype: int64

In [ ]:
# df_main.to_csv('mimic_cleaned.csv', index=False)
list(df_main.columns)